# MLflow Training Tutorial (la copie)

In [1]:
def train(labels_array, nb_epochs, nb_patience):

    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, Activation, Dropout, Flatten, Dense
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.preprocessing import image

    import mlflow
    import mlflow.tensorflow

    # Telechargement du ZIP
    from modules.loader import Loader

    loader = Loader(
        "https://stdatalake010.blob.core.windows.net/public/cifar-100.zip",
        '../datas/ZIP/',
        extraction_target='../datas/RAW/'
    )
    loader.ensure_data_loaded()

    # Extraction du jeu de donnees
    from modules.splitting import Splitting

    labels_array = ['apple', 'bee']

    TRAIN_DATA_DIR = Splitting.copie_dossiers(
        '../datas/RAW/train',
        labels_array,
        500,
        explorer=False
    )

    print(TRAIN_DATA_DIR)

    # Chargement des images
    image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
    TRAIN_IMAGE_SIZE = 32
    TRAIN_BATCH_SIZE = 64

    train_generator = image_data_generator.flow_from_directory(
        TRAIN_DATA_DIR,
        target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
        batch_size=TRAIN_BATCH_SIZE,
        class_mode='categorical',
        subset='training')
    
    validation_generator = image_data_generator.flow_from_directory(
        TRAIN_DATA_DIR, # same directory as training data
        target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
        batch_size=TRAIN_BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

    with mlflow.start_run():

        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same', input_shape=(32,32,3)))
        #Toujours à la fin
        model.add(Flatten())
        model.add(Dense(2, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        es_callback = EarlyStopping(monitor='val_loss', patience=nb_patience)
        training = model.fit(train_generator, epochs=nb_epochs, callbacks=[es_callback], validation_data=validation_generator, shuffle=False)
        
        # mlflow.log_param("labels_array", labels_array)
        # mlflow.log_param("nb_epochs", nb_epochs)
        # mlflow.log_param("nb_patience", nb_patience)

        # mlflow.log_metric("accuracy", training.history['accuracy'])
        # mlflow.log_metric("val_accuracy", training.history['val_accuracy'])
        # mlflow.log_metric("loss", training.history['loss'])
        # mlflow.log_metric("val_loss", training.history['val_loss'])

In [2]:
labels_array = ['apple', 'bee']
nb_epochs = 15
nb_patience = 4

import mlflow
mlflow.tensorflow.autolog()

train(labels_array, nb_epochs, nb_patience)

Le fichier ZIP existe déjà
Le dossier d:\simplon\datas\RAW\train_apple_bee existe déjà !
d:\simplon\datas\RAW\train_apple_bee
Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
2021/01/18 00:41:31 INFO mlflow.utils.autologging_utils: tensorflow autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow to the MLflow run with ID 'cf4b8299f71a4337890694b3afe4d231'
Epoch 1/15
 1/13 [=>............................] - ETA: 0s - loss: 52.3883 - accuracy: 0.3281WARNING:tensorflow:From D:\ana_conda\envs\sandbox\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
13/13 [==============================] - 1s 72ms/step - loss: 426.5311 - accuracy: 0.6125 - val_loss: 71.2311 - val_accuracy: 0.8350
Epoch 2/15
13/1